# 과적합(Overfitting)

실습
과적합 될 모델과 비교하기 위해 아래와 같이 기본 모델을 생성합니다.
basic_model = tf.keras.Sequential([ tf.keras.layers.Dense(256, activation = 'relu', input_shape=(word_num,)), tf.keras.layers.Dense(128, activation = 'relu'),
tf.keras.layers.Dense(1, activation= 'sigmoid')])
Copy
기본 모델을 응용해 과적합 될 모델을 생성합니다.

모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.

두 모델에 대해 손실 함수(loss function), 최적화(optimize) 알고리즘, 평가 방법(metrics)은 다음과 같이 설정합니다.

손실 함수(loss) : ‘binary_crossentropy’
최적화 알고리즘(optimizer) : ‘adam’
평가 방법(metrics): [‘accuracy’, ‘binary_crossentropy’]
Tips!
과적합 된 모델 특성상 학습 시간이 매우 오래 걸릴 수 있습니다.

In [5]:
import numpy as np
import tensorflow as tf
import pandas as pd

import logging, os
logging.disable(logging.WARNING) 

# 데이터를 전처리하는 함수

def sequences_shaping(sequences, dimension):
    
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0 
        
    return results
    
'''
1. 과적합 될 모델과 비교하기 위해 기본 모델을 
   마크다운 설명과 동일하게 생성합니다.
'''

def Basic(word_num):
    
    basic_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation = 'relu', input_shape=(word_num,)), 
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(1, activation= 'sigmoid')])
    
    return basic_model

'''
2. 기본 모델의 레이어 수와 노드 수를 자유롭게 늘려서
   과적합 될 모델을 생성합니다.
'''

def Overfitting(word_num):
    
    overfit_model = tf.keras.Sequential([
        tf.keras.layers.Dense(1024, activation = 'relu', input_shape=(word_num,)), 
        tf.keras.layers.Dense(256, activation = 'relu'),
        tf.keras.layers.Dense(256, activation = 'relu'),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(1, activation= 'sigmoid')])
    
    return overfit_model

'''
3. 두 개의 모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.

   Step01. basic_model와 overfit_model 함수를 이용해 
           두 모델을 불러옵니다.
   
   Step02. 두 모델의 손실 함수, 최적화 알고리즘, 
           평가 방법을 설정합니다.
   
   Step03. 두 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 두 모델을 학습시킵니다. 
           검증용 데이터도 설정해주세요.
           
           기본 모델은 'epochs'를 20, 
           과적합 모델은 'epochs'를 300이상으로 설정합니다.
           'batch_size'는 두 모델 모두 500으로 설정합니다.
   
   Step05. 두 모델을 테스트하고 
           binary crossentropy 값을 출력합니다. 
'''
    
word_num = 100
data_num = 25000

# Keras에 내장되어 있는 imdb 데이터 세트를 불러오고 전처리합니다.

(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words = word_num)

train_data = sequences_shaping(train_data, dimension = word_num)
test_data = sequences_shaping(test_data, dimension = word_num)

train_labels_df = pd.DataFrame(train_labels)
test_labels_df = pd.DataFrame(test_labels)

basic_model = Basic(word_num)    # 기본 모델입니다.
overfit_model = Overfitting(word_num)  # 과적합시킬 모델입니다.

basic_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'binary_crossentropy'])
overfit_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'binary_crossentropy'])

basic_model.summary()
overfit_model.summary()

basic_history = basic_model.fit(train_data, train_labels_df, epochs=20, batch_size=500, validation_data=(test_data, test_labels_df), verbose=0)
print('\n')
overfit_history = overfit_model.fit(train_data, train_labels_df, epochs=20, batch_size=500, validation_data=(test_data, test_labels_df), verbose=0)

scores_basic = basic_model.evaluate(test_data, test_labels_df,verbose=0)
scores_overfit = overfit_model.evaluate(test_data, test_labels_df, verbose=0)

print('\nscores_basic: ', scores_basic[-1])
print('scores_overfit: ', scores_overfit[-1])

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                │ (None, 256)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,881 (230.00 KB)

 Trainable params: 58,881 (230.00 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_39 (Dense)                │ (None, 1024)           │       103,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 507,905 (1.94 MB)

 Trainable params: 507,905 (1.94 MB)

 Non-trainable params: 0 (0.00 B)




scores_basic:  0.740561306476593
scores_overfit:  1.9064360857009888


In [ ]:
import numpy as np
import tensorflow as tf
from visual import *

import logging, os
logging.disable(logging.WARNING) 

# 데이터를 전처리하는 함수

def sequences_shaping(sequences, dimension):
    
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0 
        
    return results
    
'''
1. 과적합 될 모델과 비교하기 위해 기본 모델을 
   마크다운 설명과 동일하게 생성합니다.
'''

def Basic(word_num):
    
    basic_model = tf.keras.Sequential([None])
    
    return basic_model

'''
2. 기본 모델의 레이어 수와 노드 수를 자유롭게 늘려서
   과적합 될 모델을 생성합니다.
'''

def Overfitting(word_num):
    
    overfit_model = tf.keras.Sequential([None])
    
    return overfit_model

'''
3. 두 개의 모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.

   Step01. basic_model와 overfit_model 함수를 이용해 
           두 모델을 불러옵니다.
   
   Step02. 두 모델의 손실 함수, 최적화 알고리즘, 
           평가 방법을 설정합니다.
   
   Step03. 두 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 두 모델을 학습시킵니다. 
           검증용 데이터도 설정해주세요.
           
           기본 모델은 'epochs'를 20, 
           과적합 모델은 'epochs'를 300이상으로 설정합니다.
           'batch_size'는 두 모델 모두 500으로 설정합니다.
   
   Step05. 두 모델을 테스트하고 
           binary crossentropy 값을 출력합니다. 
'''

def main():
    
    word_num = 100
    data_num = 25000
    
    # Keras에 내장되어 있는 imdb 데이터 세트를 불러오고 전처리합니다.
    
    (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words = word_num)
    
    train_data = sequences_shaping(train_data, dimension = word_num)
    test_data = sequences_shaping(test_data, dimension = word_num)
    
    basic_model = None    # 기본 모델입니다.
    overfit_model = None  # 과적합시킬 모델입니다.
    
    basic_model.compile(None)
    overfit_model.compile(None)
    
    None
    None
    
    basic_history = None
    print('\n')
    overfit_history = None
    
    scores_basic = None
    scores_overfit = None
    
    print('\nscores_basic: ', scores_basic[-1])
    print('scores_overfit: ', scores_overfit[-1])
    
    Visualize([('Basic', basic_history),('Overfitting', overfit_history)])
    
    return basic_history, overfit_history

if __name__ == "__main__":
    main()